In [1]:
import math
import numpy as np
from keras.layers import (
    Dense,
    Activation,
    Dropout,
    Flatten,
    AveragePooling2D,
)
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from keras.applications import InceptionV3, Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

In [2]:
# Seed value (can actually be different for each attribution step)
seed_value= 0
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value) # tensorflow 2.x
# tf.set_random_seed(seed_value) # tensorflow 1.x

# Utils

In [3]:
# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x

get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate

# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Loading models

In [4]:
base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )
for layer in base_model.layers[:len(base_model.layers)-26]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-26:]:
    layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_resnet_original = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_resnet_original.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])
model_resnet_original.load_weights('../input/models/inception_resnet_retrained.h5')

219062272/219055592 [==============================] - 8s 0us/step


In [5]:
base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224,224,3)
    )
for layer in base_model.layers[:len(base_model.layers)-26]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-26:]:
    layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_resnet_cropped = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_resnet_cropped.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])
model_resnet_cropped.load_weights('../input/models/inception_resnet_cropped_retrained.h5')

In [6]:
base_model = InceptionV3(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_inception_original = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_inception_original.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])
model_inception_original.load_weights('../input/models/inception_v3_retrained_v2.h5')

87916544/87910968 [==============================] - 4s 0us/step


In [7]:
base_model = InceptionV3(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224,224,3)
    )
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_inception_cropped = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_inception_cropped.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])
model_inception_cropped.load_weights('../input/models/inception_v3_cropped_retrained.h5')

In [8]:
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )
for layer in base_model.layers[:len(base_model.layers)-30]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-30:]:
    layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_xception_original = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_xception_original.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])
model_xception_original.load_weights('../input/xception/inception_xception_retrained.h5')

83689472/83683744 [==============================] - 1s 0us/step


In [9]:
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224,224,3)
    )
for layer in base_model.layers[:len(base_model.layers)-30]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-30:]:
    layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_xception_cropped = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_xception_cropped.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])
model_xception_cropped.load_weights('../input/xception/inception_xception_cropped_retrained.h5')

In [10]:
lst_models = [model_resnet_original, model_resnet_cropped, model_inception_original, model_inception_cropped,
             model_xception_original, model_xception_cropped]

# Load data

In [ ]:
img_rows, img_cols = (334,334)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/train_images',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bicubic')
 
validation_generator = validation_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')

# New Approach
I will try to freeze the layers that I have freezed and unfreeze layers before

In [ ]:
for layer in model_inception_original.layers[len(model_inception_original.layers)-22:len(model_inception_original.layers)-5]:
    layer.trainable = False
for layer in model_inception_original.layers[len(model_inception_original.layers)-32:len(model_inception_original.layers)-22]:
    layer.trainable = True

In [ ]:
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("./resnet_original.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=10
batch_size=16

history = model_inception_original.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

well it's not better, I think it will only make the model overfit, so I will keep this for now

# Make Predictions

In [11]:
img_rows, img_cols = (334,334)

test_datagen = ImageDataGenerator(rescale=1./255)
 
test_generator = test_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/test_images',
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode=None,
        shuffle=False,
        interpolation='bicubic')

Found 517 images belonging to 1 classes.


In [12]:
predictions_inception_original = model_inception_original.predict(test_generator)

In [13]:
predictions_resnet_original = model_resnet_original.predict(test_generator)

In [14]:
predictions_xception_original = model_xception_original.predict(test_generator)

In [15]:
img_rows, img_cols = (224,224)

test_datagen_crop = ImageDataGenerator(rescale=1./255)
 
test_generator_crop = test_datagen_crop.flow_from_directory(
        '../input/bird-dasat/bird_dataset/test_images_cropped',
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode=None,
        shuffle=False,
        interpolation='bicubic')

Found 459 images belonging to 1 classes.


In [16]:
predictions_inception_crop = model_inception_cropped.predict(test_generator_crop)
predictions_resnet_crop = model_resnet_cropped.predict(test_generator_crop)
predictions_xception_crop = model_xception_cropped.predict(test_generator_crop)

In [17]:
filenames_orig = [x.split('/')[1][:-4] for x in test_generator.filenames]
filenames_crop = [x.split('/')[1][:-4] for x in test_generator_crop.filenames]

# Ensemble Model

## First Strategy : Max Probability

In [18]:
category = []
all_probas = []
for idx, elem in enumerate(filenames_orig):
    inception_orig = (np.max(predictions_inception_original[idx]), np.argmax(predictions_inception_original[idx]))
    resnet_orig = (np.max(predictions_resnet_original[idx]), np.argmax(predictions_resnet_original[idx]))
    xception_orig = (np.max(predictions_xception_original[idx]), np.argmax(predictions_xception_original[idx]))
    try:
        indice = filenames_crop.index(elem)
        inception_crop = (np.max(predictions_inception_crop[indice]), np.argmax(predictions_inception_crop[indice]))
        resnet_crop = (np.max(predictions_resnet_crop[indice]), np.argmax(predictions_resnet_crop[indice]))
        xception_crop = (np.max(predictions_xception_crop[indice]), np.argmax(predictions_xception_crop[indice]))
        all_proba = [inception_orig, resnet_orig, xception_orig, inception_crop, resnet_crop, xception_crop]
    except ValueError:
        all_proba = [inception_orig, resnet_orig, xception_orig]
    max_proba, cat = all_proba[0]
    for prob, label in all_proba[1:]:
        if prob > max_proba:
            max_proba = prob
            cat = label
    all_probas.append(all_proba)
    category.append(cat)
        

In [21]:
import pandas as pd
df = pd.DataFrame({'Id': filenames_orig, 'Category': category})
df.head()

,Id,Category
0,002f61512a368e4c1434eedacf609957,5
1,0247efd7b9d47d036bb4390202a13e69,10
2,0267548c2aac82fe3d7e37ae98b00bd7,17
3,030c7d18b20ee586db3b74d9966c0348,18
4,034abbbb69336b0de7c7c0f2aa1267a6,17


In [22]:
df.to_csv('./submissions_11.csv', index=False)

0.80645

## Second Strategy : Voting

In [24]:
category_ = []
def most_frequent(List): 
    return max(set(List), key = List.count)
for idx, elem in enumerate(filenames_orig):
    labels = [np.argmax(predictions_inception_original[idx]), np.argmax(predictions_resnet_original[idx]), 
             np.argmax(predictions_xception_original[idx])]
    probs = [np.max(predictions_inception_original[idx]), np.max(predictions_resnet_original[idx]),
            np.max(predictions_xception_original[idx])]
    try:
        indice = filenames_crop.index(elem)
        labels += [np.argmax(predictions_inception_crop[indice]), np.argmax(predictions_resnet_crop[indice]),
                  np.argmax(predictions_xception_crop[indice])]
        probs += [np.max(predictions_inception_crop[indice]), np.max(predictions_resnet_crop[indice]), 
                 np.max(predictions_xception_crop[indice])]                      
    except ValueError:
        pass
    if len(list(set(labels))) < len(labels):
        category_.append(most_frequent(labels))
    else:
        max_indice = probs.index(max(probs))
        category_.append(labels[max_indice])

In [29]:
df_ = pd.DataFrame({'Id': filenames_orig, 'Category': category_})
df_.head()

,Id,Category
0,002f61512a368e4c1434eedacf609957,5
1,0247efd7b9d47d036bb4390202a13e69,10
2,0267548c2aac82fe3d7e37ae98b00bd7,17
3,030c7d18b20ee586db3b74d9966c0348,18
4,034abbbb69336b0de7c7c0f2aa1267a6,17


In [30]:
df_.to_csv('./submissions_12.csv', index=False)

0.81290